In [10]:
import os
import besos
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist,DependentParameter
from besos.objectives import MeterReader,time_series_values,EPResults
from besos.optimizer import NSGAII, df_solution_to_solutions
import pandas as pd

In [11]:
# Open the IDF file
idf_directory = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/1_layer_window.idf"
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'
building = ef.get_building(idf_directory)

In [12]:
# check the version if 8.X then convert it using E+
ep.get_idf_version(building)

'9.0'

In [13]:
# get a list of the Name property of all IDF objects of class MATERIAL
[materials.Name for materials in building.idfobjects["MATERIAL"]]

['Cast Concrete (Dense)_.1',
 'Gypsum Plasterboard_.O25',
 'Brickwork Outer_.1',
 'XPS Extruded Polystyrene  - CO2 Blowing_.O792',
 'Concrete Block (Medium)_.1',
 'Gypsum Plastering_.O13']

In [14]:
## Set up evaluator
# Parameter Space
parameters = []
parameters = expand_plist( # Use helper function to make parameter list
    {'Building':                      # Name from IDF Building object
          {'North Axis':(0,359)}}     # Change orientation from North
    )

parameters.append(wwr(RangeParameter(0.1,0.9)))

zero_to_one_exclusive = RangeParameter(min_val = 0.01, max_val=0.99) # TODO change parameters

insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792', field_name='Thickness')
parameters.append(Parameter(selector=insulation,value_descriptor=zero_to_one_exclusive,name='Insulation Thickness'))  # the name is just for readability

print(parameters)

[Parameter(name='North Axis', selector=FieldSelector(field_name='North Axis', object_name='Building'), value_descriptor=RangeParameter(min=0, max=359)), Parameter(name='Window to Wall Ratio', selector=wwrSelector(), value_descriptor=RangeParameter(min=0.1, max=0.9)), Parameter(name='Insulation Thickness', selector=FieldSelector(field_name='Thickness', class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792'), value_descriptor=RangeParameter(min=0.01, max=0.99))]


In [16]:
# Objectives of our problem which is to minimize the TOTAL COMSUMPTION
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']

# problem = parameters + objectives
besos_problem = EPProblem(parameters, objectives)

# EPProblem vs EvaluatorEP = problem + building
evaluator = EvaluatorEP(besos_problem, building, out_dir='outputdir', err_dir='outputdir') 


result = evaluator([0, 0.5, 0.0792]) 
values = dict(zip(objectives, result))
for key, value in values.items():
    print(key, " :: ", "{0:.2f}".format(value/3.6e6), "kWh")

Electricity:Facility  ::  1603.62 kWh
DistrictCooling:Facility  ::  103.60 kWh
DistrictHeating:Facility  ::  6421.32 kWh
